In [160]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import random
import time

def open_browser_for_captcha():
    # 随机 User-Agent
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
    ]
    
    options = Options()
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    
    # 随机窗口大小
    window_sizes = [(1920, 1080), (1366, 768), (1440, 900), (1280, 720)]
    width, height = random.choice(window_sizes)
    
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(width, height)
    
    # 先访问 Google 检查网络状态
    print("浏览器已打开，尝试访问 Google 以检查网络...")
    driver.get("http://www.google.com")
    time.sleep(2)
    
    print("如果看到 Wi-Fi 登录页面（如 TP-LINK_7EA8），请完成认证。")
    print("认证后，按 Enter 键跳转到目标 URL。")
    print("完成后，手动完成验证码验证，再关闭浏览器窗口。")
    
    input("按 Enter 键后跳转到目标 URL...")
    url = "https://www.reuters.com/archive/2019-01/05/1/"
    driver.get(url)
    time.sleep(random.uniform(1, 3))
    
    return driver

driver = open_browser_for_captcha()

浏览器已打开，尝试访问 Google 以检查网络...
如果看到 Wi-Fi 登录页面（如 TP-LINK_7EA8），请完成认证。
认证后，按 Enter 键跳转到目标 URL。
完成后，手动完成验证码验证，再关闭浏览器窗口。


按 Enter 键后跳转到目标 URL... 


In [157]:
import pickle

cookies = driver.get_cookies()
with open("cookies.pkl", "wb") as f:
    pickle.dump(cookies, f)
print("Cookies 已保存到 cookies.pkl")
driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=55249): Max retries exceeded with url: /session/7635e71cdfdf0779e6fdac5c89433d7f/cookie (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x158772e40>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [159]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import time
import pickle
from datetime import datetime, timedelta
import random

def parse_archive_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    articles = []
    for link in soup.find_all('a', {'data-testid': 'TitleLink'}):
        href = link.get('href')
        if href.startswith("/article/"):
            articles.append("https://www.reuters.com" + href)
    return articles

def get_articles_for_date(date, cookies):
    page_no = 1
    all_articles = set()
    base_url = f"https://www.reuters.com/archive/{date.strftime('%Y-%m')}/{date.day:02d}"
    
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
    ]
    
    options = Options()
    options.add_argument("--headless")
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    
    driver = webdriver.Chrome(options=options)
    
    window_sizes = [(1920, 1080), (1366, 768), (1440, 900), (1280, 720)]
    width, height = random.choice(window_sizes)
    driver.set_window_size(width, height)
    
    driver.get("https://www.reuters.com")
    for cookie in cookies:
        driver.add_cookie(cookie)
    
    try:
        while True:
            url = f"{base_url}/{page_no}/"
            driver.get(url)
            time.sleep(random.uniform(2, 5))
            
            html = driver.page_source
            articles = parse_archive_page(html)
            
            if not articles:
                break
                
            new_articles = set(articles) - all_articles
            if not new_articles:
                break
                
            all_articles.update(new_articles)
            print(f"Date: {date.strftime('%Y%m%d')} - Page {page_no}: Found {len(new_articles)} new articles")
            page_no += 1
            
    except Exception as e:
        print(f"Error fetching {url}: {e}")
    finally:
        driver.quit()
    
    return list(all_articles)

def save_articles(date, articles):
    os.makedirs("articles_links", exist_ok=True)
    filename = f"articles_links/{date.strftime('%Y%m%d')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        for article in articles:
            f.write(f"{article}\n")
    print(f"Saved {len(articles)} articles to {filename}")

def main():
    if not os.path.exists("cookies.pkl"):
        print("请先运行前两个单元格获取 Cookies！")
        return
    
    with open("cookies.pkl", "rb") as f:
        cookies = pickle.load(f)
    print("已加载 Cookies")
    
    start_date = datetime(2019, 1, 5)
    end_date = datetime(2019, 12, 31)
    
    current_date = start_date
    while current_date <= end_date:
        articles = get_articles_for_date(current_date, cookies)
        if articles:
            save_articles(current_date, articles)
        current_date += timedelta(days=1)

main()

已加载 Cookies


Exception ignored in: <function Service.__del__ at 0x1585614e0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/selenium/webdriver/common/service.py", line 200, in __del__
  File "/opt/anaconda3/lib/python3.12/site-packages/selenium/webdriver/common/service.py", line 157, in stop
  File "/opt/anaconda3/lib/python3.12/site-packages/selenium/webdriver/common/service.py", line 137, in send_remote_shutdown_command
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 215, in urlopen
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 515, in open
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 532, in _open
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 492, in _call_chain
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 1373, in http_open
  File "/opt/anaconda3/lib/python3.12/urllib/request.py", line 1348, in do_open
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 1428, in g

KeyboardInterrupt: 

In [2]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta

def parse_archive_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    articles = []
    
    for link in soup.find_all('a', {'data-testid': 'TitleLink'}):
        href = link.get('href')
        if href.startswith("/article/"):
            articles.append("https://www.reuters.com" + href)
    
    return articles

def get_articles_for_date(date):
    base_url = f"https://www.reuters.com/archive/{date.strftime('%Y-%m')}/{date.strftime('%d')}/"
    page_no = 1
    all_articles = set()  # 使用集合避免重复
    
    while True:
        url = f"{base_url}{page_no}/"
        print(f'url={url}')
        try:
            response = requests.get(url)
            print(f'response.text={response.text}')
            if response.status_code != 200:
                break
                
            print(f'response.text={response.text}')
            articles = parse_archive_page(response.text)
            
            if not articles:  # 如果没有文章了，就停止
                break
                
            new_articles = set(articles) - all_articles
            if not new_articles:  # 如果没有新的文章了，就停止
                break
                
            all_articles.update(new_articles)
            page_no += 1
            
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            break
            
    return list(all_articles)

def save_articles_to_file(date, articles):
    # 创建目录
    os.makedirs("articles_links", exist_ok=True)
    
    # 文件名为 YYYYMMDD.txt
    filename = f"articles_links/{date.strftime('%Y%m%d')}.txt"
    
    with open(filename, 'w', encoding='utf-8') as f:
        for article in articles:
            f.write(f"{article}\n")
    print(f"Saved {len(articles)} articles to {filename}")

def main():
    # 设置日期范围
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2020, 12, 31)
    
    current_date = start_date
    while current_date <= end_date:
        print(f"Processing {current_date.strftime('%Y-%m-%d')}...")
        articles = get_articles_for_date(current_date)
        save_articles_to_file(current_date, articles)
        current_date += timedelta(days=1)

main()

Processing 2020-01-01...
url=https://www.reuters.com/archive/2020-01/01/1/
response.text=<html lang="en"><head><title>reuters.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAXHCgJtimuLgA9jROrg==','hsh':'2013457ADA70C67D6A4123E0A76873','t':'bv','qp':'','s':43909,'e':'4bc0b77305d0b6cedbc02e2bd750a7127f2bacf10545cc3b95d62dafb6bf543f','host':'geo.captcha-delivery.com','cookie':'82w6OquqIONe9dIw23XTOI7c0nLc_q7GdxIxh09EpY3SZc8q3FK8Z6UJnsEOfFWMvGULe8X8d0tA0pEjUAgJcjRKqUyU0YLP4xBj0khuowiimk~5BkicSwp8PZedwos0'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>
Saved 0 articles to articles_links/20200101.txt
Processing 2020-01-02...
url=https://www.reuters.com/archive/2020-01/02/1/
response.text=<html lang="en"><head><title>reute

KeyboardInterrupt: 